## Fisher Score - chi-square implementation in sklearn

Compute chi-squared stats between each non-negative feature and class. 

- This score should be used to evaluate categorical variables in a classification task.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories. I explained this in more detail the introductory lecture of this section.

I will demonstrate how to select features using Fisher score using the titanic dataset from Kaggle.

In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile

In [3]:
import os
#os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\titanic')
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\Titanic')

In [4]:
# load dataset
data = pd.read_csv('train.csv')
data.shape

(891, 12)

In [13]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
## Chi Square Test
f_score = chi2(np.array(data['Survived']).reshape(-1, 1),np.array(data['Pclass']).reshape(-1, 1))
f_score

(array([63.3962456]), array([1.7126956e-14]))

In [5]:
# the categorical variables in the titanic are PClass, Sex and Embarked
# first I will encode the labels of the categories into numbers

# for Sex / Gender
data['Sex'] = np.where(data.Sex == 'male', 1, 0)

# for Embarked
ordinal_label = {k: i for i, k in enumerate(data['Embarked'].unique(), 0)}
data['Embarked'] = data['Embarked'].map(ordinal_label)

# PClass is already ordinal

In [6]:
data['Age']=data['Age'].fillna(0)
data['Cabin']=data['Cabin'].fillna('NotAvailable')

In [7]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Important

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [5]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[['Pclass', 'Sex', 'Embarked']],
    data['Survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((623, 3), (268, 3))

In [24]:
# calculate the chi2 p_value between each of the variables
# and the target
# it returns 2 arrays, one contains the F-Scores which are then 
# evaluated against the chi2 distribution to obtain the pvalue
# the pvalues are in the second array, see below

f_score = chi2(X_train.fillna(0), y_train)
f_score

(array([21.61080949, 63.55447864, 11.83961845]),
 array([3.33964360e-06, 1.55992554e-15, 5.79837058e-04]))

In [25]:
# let's add the variable names and order it for clearer visualisation

pvalues = pd.Series(f_score[1])
pvalues.index = X_train.columns
pvalues.sort_values(ascending=False)

Embarked    5.798371e-04
Pclass      3.339644e-06
Sex         1.559926e-15
dtype: float64

Keep in mind, that contrarily to MI, where we were interested in the higher MI values, for Fisher score, the smaller the p_value, the more significant the feature is to predict the target, in this case Survival in the titanic.

Then, from the above data, Sex is the most important feature, then PClass then Embarked. 

**Note**
One thing to keep in mind when using Fisher score or univariate selection methods, is that in very big datasets, most of the features will show a small p_value, and therefore look like they are highly predictive. This is in fact an effect of the sample size. So care should be taken when selecting features using these procedures. An ultra tiny p_value does not highlight an ultra-important feature, it rather indicates that the dataset contains too many samples. 

Finally, in this demonstration, I used chi2 over 3 categorical variables only. If the dataset contained several categorical variables, we could then combine this procedure with SelectKBest or SelectPercentile, as I did in the previous lecture.

In [12]:
contingency_table = pd.crosstab(
    data['Pclass'],
    data['Survived'],
    margins = True
)
contingency_table

Survived,0,1,All
Pclass,,,
1,80,136,216
2,97,87,184
3,372,119,491
All,549,342,891


In [14]:
contingency_table.iloc[0][0:2].values

array([ 80, 136], dtype=int64)

In [15]:
f_obs = np.array([contingency_table.iloc[0][0:2].values,
                  contingency_table.iloc[1][0:2].values,
                 contingency_table.iloc[2][0:2].values])
f_obs

array([[ 80, 136],
       [ 97,  87],
       [372, 119]], dtype=int64)

In [16]:
from scipy import stats
stats.chi2_contingency(f_obs)

(102.88898875696056,
 4.549251711298793e-23,
 2,
 array([[133.09090909,  82.90909091],
        [113.37373737,  70.62626263],
        [302.53535354, 188.46464646]]))

That is all for this lecture, I hope you enjoyed it and see you in the next one!

In [27]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(X_train,y_train)

C:\Users\Administrator\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
LR.coef_[0]

array([-0.81507635, -2.3450179 ,  0.34893139])

In [30]:
pvalues = pd.Series(LR.coef_[0])
pvalues.index = X_train.columns
pvalues.sort_values(ascending=False)

Embarked    0.348931
Pclass     -0.815076
Sex        -2.345018
dtype: float64